Import all necessary libraries and install everything you need for training:

In [1]:
# install the libraries necessary for data wrangling, prediction and result analysis
import json
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, f1_score,precision_score, recall_score
import torch
from numba import cuda
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [2]:
# Install transformers
# (this needs to be done on Kaggle each time you start the session)
#!pip install -q transformers

# Install the simpletransformers
#!pip install -q simpletransformers
from simpletransformers.classification import ClassificationModel

In [3]:
# Install wandb
#!pip install -q wandb
import wandb

# Login to wandb
wandb.login()

In [5]:
# Clean the GPU cache

cuda.select_device(0)
cuda.close()
cuda.select_device(0)
torch.cuda.empty_cache()


### Import the data

In [6]:
# MT-GINCO downcast set
train_df = pd.read_csv("data-splits/MT-GINCO-downcast-train.csv", index_col = 0)
dev_df = pd.read_csv("data-splits/MT-GINCO-downcast-dev.csv", index_col = 0)
test_df = pd.read_csv("data-splits/MT-GINCO-downcast-test.csv", index_col = 0)

print("GINCO train shape: {}, Dev shape: {}, Test shape: {}.".format(train_df.shape, dev_df.shape, test_df.shape))

GINCO train shape: (601, 2), Dev shape: (201, 2), Test shape: (200, 2).


In [8]:
train_df.head()

,text,labels
324,"Ja tudi meni se zdi grdo, da moram razlagati z...",Forum
253,Bo kolesarjenje postalo del zimskih športov? <...,News/Reporting
674,Teptanje ustave namesto prevzema odgovornosti ...,Opinion/Argumentation
600,Cena odtisa za Canon i990 <p/> Zdaj smo končno...,Opinion/Argumentation
460,Veronikin tek je idealna priložnost za priprav...,Promotion


## Training and saving

We will use the multilingual XLM-RoBERTa model
https://huggingface.co/xlm-roberta-base

In [9]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [7]:
# Create a list of labels
LABELS = train_df.labels.unique().tolist()
print(LABELS)

['Forum', 'News/Reporting', 'Opinion/Argumentation', 'Promotion', 'Information/Explanation', 'List of Summaries/Excerpts', 'Other', 'Instruction', 'Legal/Regulation']


In [11]:
# Initialize Wandb
wandb.init(project="GINCO-hyperparameter-search", name="GINCO-downcast-hyperparameter-search")

We will use the same hyperparameters as for the GINCO-downcast model.

Train the model and save it.

In [15]:
# Create a TransformerModel
roberta_base_model = ClassificationModel(
        "xlmroberta", "xlm-roberta-base",
        num_labels=len(LABELS),
        use_cuda=True,
        args= {
            "overwrite_output_dir": True,
            "num_train_epochs": 15,
            "train_batch_size":8,
            "learning_rate": 1e-5,
            "labels_list": LABELS,
            # The following parameters are commented out because I want to save the model
            #"no_cache": True,
            # Disable no_save: True if you want to save the model
            #"no_save": True,
            "max_seq_length": 512,
            "save_steps": -1,
            # Only the trained model will be saved - to prevent filling all of the space
            "save_model_every_epoch":False,
            "wandb_project": 'GINCO-hyperparameter-search',
            "silent": True,
            }
        )

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [16]:
# Train the model
roberta_base_model.train_model(train_df)

/home/tajak/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


(1460, 1.138362426320984)

In [8]:
# Save the trained model to Wandb
run = wandb.init(project="GINCO-hyperparameter-search", entity="tajak", name="saving-trained-model")
trained_model_artifact = wandb.Artifact("MT-GINCO-downcast-classifier", type="model", description="a model trained on the English (MT) GINCO dataset with the a downcasted set of labels. There are 9 labels: ['Forum', 'News/Reporting', 'Opinion/Argumentation', 'Promotion', 'Information/Explanation', 'List of Summaries/Excerpts', 'Other', 'Instruction', 'Legal/Regulation'].")
trained_model_artifact.add_dir("outputs")
run.log_artifact(trained_model_artifact)

wandb: Adding directory to artifact (./outputs)... Done. 2.7s
